> # AIDA with Transfer Learning

# Imports

In [1]:
import pandas as pd
import numpy as np
import os
import math
import seaborn as sns

import urllib.request
from urllib.parse import urlparse

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#import tensorflow_addons as tfa

#https://machinelearningmastery.com/how-to-use-transfer-learning-when-developing-convolutional-neural-network-models/
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.xception import Xception
from keras.models import Model
from keras import metrics
from keras.callbacks import ModelCheckpoint, TensorBoard
from numba import cuda

import sklearn.model_selection as skms
from sklearn.utils import class_weight

#from wcs.google import google_drive_share
import urllib.request
from urllib.parse import urlparse

#from google.colab import drive
#import src.helper.helper as hlp
#import src.helper.const as const
import datetime as dt
import time

import warnings
warnings.simplefilter(action='ignore')
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

# Configuration

In [2]:
# Config
USE_MOST_POPULAR = True
NUM_MOST_POPULAR = 50000

DP_TFDATA = "Data pipeline using tf.data"
DP_IMGGEN = "Data pipeline using tf.keras.ImageGenerator"
DP = DP_TFDATA

LR = 1e-5 # Keep it small when transfer learning
BATCH_SIZE = 64
EPOCHS = 5
AUTOTUNE = tf.data.experimental.AUTOTUNE  # Adapt preprocessing and prefetching dynamically to reduce GPU and CPU idle time
SHUFFLE_BUFFER_SIZE = 1024 # Shuffle the training data by a chunck of 1024 observations

IMG_DIMS = [299, 299]
IMG_CHANNELS = 3  # Keep RGB color channels to match the input format of the model
LABEL_COLS = ['Action', 'Adventure', 'Animation', 'Comedy', 'Crime',
              'Documentary', 'Drama', 'Family', 'Fantasy', 'History', 'Horror',
              'Music', 'Mystery', 'Romance', 'Science Fiction', 'TV Movie',
              'Thriller', 'War', 'Western']

DIR = './'
DATA_DIR_POSTER = DIR + '../data/raw/posters_v3/'
DATA_DIR_INTERIM = DIR + "../data/interim/"
DATA_DIR_RAW = DIR + "../data/raw/"
MODEL_DIR = DIR + "../models/"
BASE_DIR = DIR
IMAGES_DIR = DATA_DIR_POSTER
SEED = 42
TENSORBOARD_LOGDIR = DIR + "tensorboard_logs/scalars/"

pd.set_option('display.max_colwidth', None)

In [3]:
    import google.colab as colab
    colab.drive.mount('/content/drive')

Mounted at /content/drive


# Create model

In [10]:
from keras.applications.inception_resnet_v2 import preprocess_input as preprocess_input_inc_res
from keras.applications.xception import preprocess_input as preprocess_input_xception
from keras import backend as K
def model_create():
    base_inc_res = InceptionResNetV2(
        include_top=False, 
        weights='imagenet', 
        pooling='avg', 
        input_shape=(299,299,3))
    base_inc_res.trainable = False
    base_xception = Xception(include_top=False, weights='imagenet', pooling='avg', input_shape=(299, 299, 3))
    base_xception.trainable = False
    
    inputs = keras.Input(shape=(299,299,3))
    x1 = preprocess_input_inc_res(inputs)
    x1 = base_inc_res(x1)
    x1 = layers.BatchNormalization()(x1)
    
    x2 = preprocess_input_xception(inputs)
    x2 = base_xception(x2)
    x2 = layers.BatchNormalization()(x2)
    
    x = K.concatenate([x1, x2])
    x = layers.Dense(1024, activation='relu')(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dropout(0.25)(x)
    x = layers.Dense(19, activation='sigmoid')(x)
    
    model_name = "InceptionResNetV2_Customized"
    return keras.Model(inputs=inputs, outputs=x, name=model_name)
    return model, model_name

model_test = model_create()

In [11]:
model_test.summary()

Model: "InceptionResNetV2_Customized"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
tf.math.truediv_6 (TFOpLambda)  (None, 299, 299, 3)  0           input_12[0][0]                   
__________________________________________________________________________________________________
tf.math.truediv_7 (TFOpLambda)  (None, 299, 299, 3)  0           input_12[0][0]                   
__________________________________________________________________________________________________
tf.math.subtract_6 (TFOpLambda) (None, 299, 299, 3)  0           tf.math.truediv_6[0][0]          
_______________________________________________________________________

In [13]:
model_test.save("/content/drive/MyDrive/Colab Notebooks/InceptionResNetV2_corrected.hd5")

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/InceptionResNetV2_corrected.hd5/assets
